In [30]:
import mdtraj as md
import numpy as np
from copy import deepcopy

In [64]:
def change_bond(pdb, fix_atom, flex_atom, new_length, frame=0, inplace=False):
    """
    Changes the bond length between fix_atom (kept fixed) and flex_atom (flexible atom)
    to new_length. 
    
    Parameters
    ---------
    pdb : md.Trajectory 
        A trajectory
    fix_atom, flex_atom : int
        Integer index (0-based) of the atom to keep fixed and the atom to move
        respectively. 
    new_length : float
        The required bond length
    frame : int
        the frame of the trajectory
    inplace : bool
        whether to act on pdb (true) or return a deep copy (false)
    
    Returns
    -------
    md.Trajectory
        A new trajectory object with the bond changed
    
    Notes
    -----
    If you adjust a bond with more than one bond - you will affect more than just one bond
    length! Use with caution. 
    """
    
    old_length = md.compute_distances(pdb, [[fix_atom, flex_atom]])[0][0]
    scale = new_length/old_length

    # get coordinates or atoms
    xyz = pdb.xyz
    r1 = xyz[frame, fix_atom, :]
    r2 = xyz[frame, flex_atom, :]
    
    # calculate direction vector and scale to get new coordinates
    del_r_scaled = (r2-r1)*scale
    new_r2 = r1 + del_r_scaled
    
    # operate in place? 
    if inplace:
        new_pdb = pdb
    else:
        new_pdb = deepcopy(pdb)
    
    # adjust coordinates
    new_pdb.xyz[frame, flex_atom, :] = new_r2
    
    # check 
    actual_new_length = md.compute_distances(new_pdb, [[fix_atom, flex_atom]])[0][0]
    assert np.abs(actual_new_length-new_length)<1e-6, "New bond length is not the same as 'new_length'"
    
    new_fix_atom = new_pdb.xyz[frame, fix_atom, :]
    old_fix_atom = pdb.xyz[frame, fix_atom, :]
    assert np.allclose(new_fix_atom, old_fix_atom), "The fix_atom has changed position"
    
    return new_pdb

In [65]:
import pandas as pd


def get_dataframe(pdb, frame=0):

    df, _ =  pdb.top.to_dataframe()
    df = pd.concat([df, pd.DataFrame(pdb.xyz[frame], columns=['x', 'y', 'z'])], axis=1)
    return df

Load some pdb (pentapeptide in implicit solvent used here)

pdb availabe from the folks at FU Berlin, via `mdshare` [here](https://github.com/markovmodel/mdshare)

In [74]:
pdb = md.load('data/pentapeptide-impl-solv.pdb')

# define atom for the bond
atom_1, atom_2 = 0, 1
new_length = 0.15 # in nm

Change the bond with the function

In [75]:
new_pdb = change_bond(pdb, atom_1, atom_2, new_length)

Save as a new pdb

In [76]:
new_pdb.save_pdb('data/new_pdb.pdb')

Double check the correct atoms have move:

In [77]:
old_df = get_dataframe(pdb)
new_df = get_dataframe(new_pdb)


In [78]:
old_df.loc[old_df.index.isin([atom_1, atom_2]), :]

,serial,name,element,resSeq,resName,chainID,segmentID,x,y,z
0,1,N,N,1,TRP,0,,2.108,2.666,1.701
1,2,H,H,1,TRP,0,,2.059,2.724,1.635


In [79]:
new_df.loc[new_df.index.isin([atom_1, atom_2]), :]

,serial,name,element,resSeq,resName,chainID,segmentID,x,y,z
0,1,N,N,1,TRP,0,,2.108000,2.666000,1.701000
1,2,H,H,1,TRP,0,,2.034941,2.752478,1.602594


From pymol: 


old: 
<img src="data/old_bond.png" alt="Drawing" style="width: 600px;"/>
new:
<img src="data/new_bond.png" alt="Drawing" style="width: 600px;"/>